In [11]:
!pip install gensim nltk

In [3]:
import gensim
gensim.__version__

'4.4.0'

In [4]:
from gensim.models import Word2Vec, KeyedVectors

In [5]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

In [6]:
w2v['king']

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

In [7]:
import pandas as pd
import os 

path = os.path.join('sms_spam_data', 'SMSSpamCollection')
df = pd.read_csv(path, sep='\t', names=['target', 'message'])
df.head()

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.shape

(5572, 2)

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
from nltk import word_tokenize

In [16]:
import re

corpus = []

for i in range(len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review_tokens = word_tokenize(review.lower())
    review.lower()

    review = [lemmatizer.lemmatize(token) for token in review_tokens]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [18]:
corpus[1]

'ok lar joking wif u oni'

In our corpus as we are filtering the text with regular expression, anything apart from a-z and A-Z won't be considered

In [19]:
for i in range(len(corpus)):
    if len(corpus[i]) < 1:
        print(f"Empty data after consition: {df['message'][i]}")
        print(f"index {i}")

Empty data after consition: 645
index 1612
Empty data after consition: :) 
index 3376
Empty data after consition: :-) :-)
index 4824


### Training Custum Word2Vec

In [20]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
words = []

for sent in corpus:
    sent_token = sent_tokenize(sent)
    for s in sent_token:
        words.append(simple_preprocess(sent))

In [22]:
words[:5]

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'in',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'to',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'],
 ['nah',
  'don',
  'think',
  'he',
  'go',
  'to',
  'usf',
  'he',
  'life',
  'around',
  'here',
  'though']]

In [23]:
custom_w2v = Word2Vec(sentences=words)

In [24]:
# To Get All the Vocabulary
# custom_w2v.wv.index_to_key

# Indexed
custom_w2v.wv.index_to_key[:5]

['to', 'you', 'the', 'and', 'it']

In [25]:
custom_w2v.corpus_count

5569

In [26]:
custom_w2v.epochs

5

In [27]:
custom_w2v.wv.similar_by_word('good')

[('hope', 0.9987856149673462),
 ('day', 0.9986543655395508),
 ('all', 0.9985660314559937),
 ('happy', 0.9985291361808777),
 ('great', 0.998526930809021),
 ('morning', 0.9985029101371765),
 ('well', 0.9984906911849976),
 ('where', 0.9984448552131653),
 ('wa', 0.9983952641487122),
 ('thing', 0.9983922243118286)]

In [28]:
custom_w2v.wv['good'].shape

(100,)

In [29]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([custom_w2v.wv[word] for word in doc if word in custom_w2v.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)

In [30]:
!pip install tqdm

In [31]:
from tqdm import tqdm

In [32]:
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5569 [00:00<?, ?it/s]/opt/conda/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.12/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5569/5569 [00:00<00:00, 13183.14it/s]


In [33]:
len(X)

5569

In [34]:
X[:2]

[array([-0.08355109,  0.24844213,  0.10884197,  0.09398454,  0.14190392,
        -0.46902296,  0.16707529,  0.5016658 , -0.27339935, -0.13520204,
        -0.28633076, -0.38179094, -0.1122661 ,  0.09415754,  0.19439428,
        -0.0859109 ,  0.09274013, -0.3036643 , -0.12387542, -0.5806284 ,
         0.15460698,  0.23316173,  0.121745  , -0.2762162 , -0.07226048,
         0.0605105 , -0.27200833, -0.27633435, -0.26229623,  0.10374652,
         0.2772873 ,  0.01557377,  0.15965165, -0.23501843, -0.16906087,
         0.30016762,  0.11422792, -0.19878002, -0.04366549, -0.46127355,
         0.11825656, -0.27839556, -0.10910421, -0.04561473,  0.24732979,
        -0.08416722, -0.20550299, -0.0319486 ,  0.14376287,  0.23735866,
         0.15258452, -0.26280266, -0.12583992, -0.01990926, -0.04221523,
         0.18148655,  0.14801647,  0.0734077 , -0.24619295,  0.20122477,
        -0.03917527,  0.05712701, -0.04825367, -0.06425735, -0.2970446 ,
         0.21736002,  0.09856321,  0.2191281 , -0.3

In [35]:
len(X[0])

100

In [39]:
## Dependent Features
## Output Features
y = df[list(map(lambda x: len(x) > 0, corpus))]
y[:5]

,target,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [40]:
y = pd.get_dummies(y['target'])
y[:5]

,ham,spam
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False


In [41]:
y = y.iloc[:,0].values
y[:5]

array([ True,  True, False,  True,  True])

In [42]:
X[0].reshape(1,-1).shape

(1, 100)

In [70]:
## this is the final independent features
df=pd.DataFrame()
for i in range(0,len(X)):
    df=pd.concat([df, pd.DataFrame(X[i].reshape(1,-1))],ignore_index=True)

/tmp/ipykernel_743/2704690250.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df=pd.concat([df, pd.DataFrame(X[i].reshape(1,-1))],ignore_index=True)


In [71]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.083551,0.248442,0.108842,0.093985,0.141904,-0.469023,0.167075,0.501666,-0.273399,-0.135202,...,0.265961,0.203251,-0.044628,0.093050,0.399818,0.276059,0.187874,-0.246156,0.131937,-0.069053
1,-0.070169,0.216485,0.093327,0.084264,0.123394,-0.416151,0.146296,0.439921,-0.230260,-0.108484,...,0.232728,0.171316,-0.043650,0.078344,0.344308,0.230925,0.154745,-0.219743,0.106119,-0.056090
2,-0.054762,0.271693,0.118975,0.119335,0.166403,-0.496622,0.168722,0.515891,-0.295713,-0.136579,...,0.265900,0.217310,-0.046098,0.092391,0.398410,0.268722,0.174906,-0.269167,0.155947,-0.074937
3,-0.118912,0.333200,0.143987,0.128059,0.179890,-0.633076,0.229290,0.678857,-0.371369,-0.175681,...,0.365823,0.273616,-0.061261,0.135157,0.533247,0.370370,0.246797,-0.340412,0.167563,-0.080578
4,-0.112441,0.280812,0.131958,0.112745,0.158027,-0.547725,0.192466,0.580688,-0.321735,-0.159541,...,0.322715,0.227880,-0.044997,0.116225,0.459729,0.317484,0.217655,-0.297768,0.150623,-0.078481


In [72]:
df['output'] = y

In [73]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,output
0,-0.083551,0.248442,0.108842,0.093985,0.141904,-0.469023,0.167075,0.501666,-0.273399,-0.135202,...,0.203251,-0.044628,0.093050,0.399818,0.276059,0.187874,-0.246156,0.131937,-0.069053,True
1,-0.070169,0.216485,0.093327,0.084264,0.123394,-0.416151,0.146296,0.439921,-0.230260,-0.108484,...,0.171316,-0.043650,0.078344,0.344308,0.230925,0.154745,-0.219743,0.106119,-0.056090,True
2,-0.054762,0.271693,0.118975,0.119335,0.166403,-0.496622,0.168722,0.515891,-0.295713,-0.136579,...,0.217310,-0.046098,0.092391,0.398410,0.268722,0.174906,-0.269167,0.155947,-0.074937,False
3,-0.118912,0.333200,0.143987,0.128059,0.179890,-0.633076,0.229290,0.678857,-0.371369,-0.175681,...,0.273616,-0.061261,0.135157,0.533247,0.370370,0.246797,-0.340412,0.167563,-0.080578,True
4,-0.112441,0.280812,0.131958,0.112745,0.158027,-0.547725,0.192466,0.580688,-0.321735,-0.159541,...,0.227880,-0.044997,0.116225,0.459729,0.317484,0.217655,-0.297768,0.150623,-0.078481,True


In [74]:
df.isna().sum()

0         12
1         12
2         12
3         12
4         12
          ..
96        12
97        12
98        12
99        12
output     0
Length: 101, dtype: int64

In [75]:
sum(df.isna().sum())

1200

In [76]:
df.isna().sum(axis=1)

0       0
1       0
2       0
3       0
4       0
       ..
5564    0
5565    0
5566    0
5567    0
5568    0
Length: 5569, dtype: int64

In [77]:
df[df.isna().sum(axis=1) > 0]

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,output
451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [78]:
df.dropna(axis=0, inplace = True)

In [79]:
sum(df.isna().sum())

0

In [82]:
X = df.drop(['output'], axis=1)
y = df['output']

In [84]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.083551,0.248442,0.108842,0.093985,0.141904,-0.469023,0.167075,0.501666,-0.273399,-0.135202,...,0.265961,0.203251,-0.044628,0.093050,0.399818,0.276059,0.187874,-0.246156,0.131937,-0.069053
1,-0.070169,0.216485,0.093327,0.084264,0.123394,-0.416151,0.146296,0.439921,-0.230260,-0.108484,...,0.232728,0.171316,-0.043650,0.078344,0.344308,0.230925,0.154745,-0.219743,0.106119,-0.056090
2,-0.054762,0.271693,0.118975,0.119335,0.166403,-0.496622,0.168722,0.515891,-0.295713,-0.136579,...,0.265900,0.217310,-0.046098,0.092391,0.398410,0.268722,0.174906,-0.269167,0.155947,-0.074937
3,-0.118912,0.333200,0.143987,0.128059,0.179890,-0.633076,0.229290,0.678857,-0.371369,-0.175681,...,0.365823,0.273616,-0.061261,0.135157,0.533247,0.370370,0.246797,-0.340412,0.167563,-0.080578
4,-0.112441,0.280812,0.131958,0.112745,0.158027,-0.547725,0.192466,0.580688,-0.321735,-0.159541,...,0.322715,0.227880,-0.044997,0.116225,0.459729,0.317484,0.217655,-0.297768,0.150623,-0.078481


In [85]:
y.head()

0     True
1     True
2    False
3     True
4     True
Name: output, dtype: bool

In [93]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

# Model Training and Evaluation

In [94]:
from sklearn.ensemble import RandomForestClassifier

In [95]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [96]:
y_pred=classifier.predict(X_test)

In [97]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.9694244604316546


In [98]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.92      0.87      0.89       163
        True       0.98      0.99      0.98       949

    accuracy                           0.97      1112
   macro avg       0.95      0.93      0.94      1112
weighted avg       0.97      0.97      0.97      1112

